# Trajectory picture

In [1]:
import os
import ROOT

Welcome to JupyROOT 6.16/00


In [2]:
def simulation():
    parfile = "output/traj.para.root"
    outfile = "output/traj.simu.root"

    ROOT.ROOT.EnableThreadSafety()
    ROOT.FairLogger.GetLogger().SetLogVerbosityLevel("LOW")
    ROOT.FairLogger.GetLogger().SetLogScreenLevel("ERROR")

    vmcworkdir = os.environ["VMCWORKDIR"]
    os.environ["GEOMPATH"] = vmcworkdir + "/geometry"
    os.environ["CONFIG_DIR"] = vmcworkdir + "/gconfig"
    os.environ["PHYSICSLIST"] = "QGSP_INCLXX_HP"

    # Initialize Simulation
    run = ROOT.FairRunSim()
    run.SetName("TGeant4")
    run.SetStoreTraj(True)
    run.SetMaterials("media_r3b.geo")

    # Output
    output = ROOT.FairRootFileSink(outfile)
    run.SetSink(output)

    # Primary Generator
    generator = ROOT.FairPrimaryGenerator()
    box = ROOT.FairBoxGenerator(2112, 1)
    box.SetXYZ(0, 0, 0.0)
    box.SetThetaRange(0.0, 1.0)
    box.SetPhiRange(0.0, 360.0)
    box.SetEkinRange(0.6, 0.6)
    generator.AddGenerator(box)
    run.SetGenerator(generator)

    # Geometry
    cave = ROOT.R3BCave("Cave")
    cave.SetGeometryFileName("r3b_cave_vacuum.geo")
    run.AddModule(cave)

    doubleplane = 30
    neuland_position = ROOT.TGeoTranslation(
        0.0, 0.0, 15 * 100 + doubleplane * 10.0 / 2.0
    )
    neuland = ROOT.R3BNeuland(doubleplane, neuland_position)
    run.AddModule(neuland)

    # Prepare to run
    run.Init()
    ROOT.TVirtualMC.GetMC().SetRandom(ROOT.TRandom3(1337))
    ROOT.TVirtualMC.GetMC().SetMaxNStep(100000)

    # Runtime Database
    rtdb = run.GetRuntimeDb()
    parout = ROOT.FairParRootFileIo(True)
    parout.open(parfile)
    rtdb.setOutput(parout)
    rtdb.saveOutput()

    run.Run(20)

In [ ]:
simulation()

In [4]:
f = ROOT.TFile.Open("output/traj.simu.root")
zoffset = 1500

alldata = []
allpoints = []
for e in f.evt:
    data = []
    for i, track in enumerate(e.GeoTracks):
        datum = {}
        particle = track.GetParticle()

        lv = ROOT.TLorentzVector()
        particle.Momentum(lv)
        ekin = (lv.E() - particle.GetMass()) * 1000

        # print(i, particle.GetPdgCode(), ekin, track.GetNpoints())
        datum["pdg"] = particle.GetPdgCode()
        datum["name"] = track.GetName()
        datum["ekin"] = ekin

        tr = []

        for n in range(track.GetNpoints()):
            # Check bounds
            if (
                abs(track.GetPoint(n)[2] - (zoffset + 150)) > 300 / 2 * 1.1
                or abs(track.GetPoint(n)[1]) > 250 / 2 * 1.1
                or abs(track.GetPoint(n)[0]) > 250 / 2 * 1.1
            ):
                continue
            tr.append(
                (
                    track.GetPoint(n)[0] / 10,
                    track.GetPoint(n)[1] / 10,
                    (track.GetPoint(n)[2] - zoffset) / 10,
                )
            )
        datum["track"] = tr
        data.append(datum)
    alldata.append(data)

    points = []
    for p in e.NeulandPoints:
        pos = p.GetPosition()
        points.append(
            (pos.X() / 10, pos.Y() / 10, (pos.Z() - zoffset) / 10, p.GetLightYield())
        )
    allpoints.append(points)

In [5]:
import palettable
# colors = palettable.cartocolors.qualitative.Bold_10.colors
colors = palettable.colorbrewer.qualitative.Set1_9.colors
defaultcolor = colors[8]

particles = {}
for data in alldata:
    for datum in data:
        particles[datum["pdg"]] = {"name": datum["name"], "color": defaultcolor}

print(particles)

{2112: {'name': 'neutron', 'color': [153, 153, 153]}, 1000020040: {'name': 'XXX', 'color': [153, 153, 153]}, 1000010020: {'name': 'XXX', 'color': [153, 153, 153]}, 11: {'name': 'e-', 'color': [153, 153, 153]}, -12: {'name': 'nu_e_bar', 'color': [153, 153, 153]}, 2212: {'name': 'proton', 'color': [153, 153, 153]}, 1000060120: {'name': 'XXX', 'color': [153, 153, 153]}, 22: {'name': 'gamma', 'color': [153, 153, 153]}, 1000040090: {'name': 'XXX', 'color': [153, 153, 153]}, 1000010030: {'name': 'XXX', 'color': [153, 153, 153]}, 1000030060: {'name': 'XXX', 'color': [153, 153, 153]}, 1000050110: {'name': 'XXX', 'color': [153, 153, 153]}, 1000020030: {'name': 'XXX', 'color': [153, 153, 153]}, 1000050100: {'name': 'XXX', 'color': [153, 153, 153]}, 1000060110: {'name': 'XXX', 'color': [153, 153, 153]}, -11: {'name': 'e+', 'color': [153, 153, 153]}, 1000030070: {'name': 'XXX', 'color': [153, 153, 153]}, 1000070120: {'name': 'XXX', 'color': [153, 153, 153]}, 211: {'name': 'pi+', 'color': [153, 153

In [6]:
particles[11]["name"] = "$e^-$"
particles[-11]["name"] = "$e^+$"
particles[22]["name"] = "$\gamma$"
particles[-13]["name"] = "$\\mu^+$"
particles[12]["name"] = "$\\nu_e$"
particles[-12]["name"] = "$\\bar{\\nu}_e$"
particles[14]["name"] = "$\\nu_\mu$"
particles[-14]["name"] = "$\\bar{\\nu}_\mu$"
particles[211]["name"] = "$\pi^+$"
particles[-211]["name"] = "$\pi^-$"
particles[111]["name"] = "$\pi^0$"

particles[2112]["color"] = colors[0]
particles[2212]["color"] = colors[1]
particles[1000010020]["color"] = colors[1]
particles[1000010030]["color"] = colors[1]
particles[22]["color"] = colors[2]
particles[11]["color"] = colors[3]
particles[-11]["color"] = colors[3]
particles[1000020030]["color"] = colors[6]
particles[1000020040]["color"] = colors[6]
particles[1000020060]["color"] = colors[6]
particles[12]["color"] = colors[4]
particles[-12]["color"] = colors[4]
particles[-13]["color"] = colors[7]
particles[14]["color"] = colors[4]
particles[-14]["color"] = colors[4]
particles[211]["color"] = colors[5]
particles[-211]["color"] = colors[5]
particles[111]["color"] = colors[5]

particles[13] = {"name": "$\\mu^-$", "color": colors[7]}

print(particles)

{2112: {'name': 'neutron', 'color': [228, 26, 28]}, 1000020040: {'name': 'XXX', 'color': [166, 86, 40]}, 1000010020: {'name': 'XXX', 'color': [55, 126, 184]}, 11: {'name': '$e^-$', 'color': [152, 78, 163]}, -12: {'name': '$\\bar{\\nu}_e$', 'color': [255, 127, 0]}, 2212: {'name': 'proton', 'color': [55, 126, 184]}, 1000060120: {'name': 'XXX', 'color': [153, 153, 153]}, 22: {'name': '$\\gamma$', 'color': [77, 175, 74]}, 1000040090: {'name': 'XXX', 'color': [153, 153, 153]}, 1000010030: {'name': 'XXX', 'color': [55, 126, 184]}, 1000030060: {'name': 'XXX', 'color': [153, 153, 153]}, 1000050110: {'name': 'XXX', 'color': [153, 153, 153]}, 1000020030: {'name': 'XXX', 'color': [166, 86, 40]}, 1000050100: {'name': 'XXX', 'color': [153, 153, 153]}, 1000060110: {'name': 'XXX', 'color': [153, 153, 153]}, -11: {'name': '$e^+$', 'color': [152, 78, 163]}, 1000030070: {'name': 'XXX', 'color': [153, 153, 153]}, 1000070120: {'name': 'XXX', 'color': [153, 153, 153]}, 211: {'name': '$\\pi^+$', 'color': [2

In [20]:
import math
for en in range(20):
    out = """\\documentclass{scrartcl}
    \\usepackage{xcolor, tikz}
    \\pagestyle{empty}
    """

    cl = set([abs(datum["pdg"]) for datum in alldata[en]])
    for pdg in cl:
        out += "\\definecolor{pdg%s}{RGB}{%d,%d,%d}\n" % (str(abs(pdg)), *particles[pdg]["color"])

    out += """
    \\begin{document}
    \\begin{tikzpicture}[scale=0.4] %\columnwidth/252.0pt]
    \\draw[step=0.5,very thin,gray] (0,-12.5) grid (30,6.25);
    \\draw[step=0.5,very thin,gray] (5.25,6.25) grid (30,12.5);
    """

    for point in allpoints[en]:
        out += "\\draw (%f,%f) circle (%f);\n" % (point[2], point[1], 0.1*math.log(point[3]) )

    for datum in alldata[en]:
        out += "\\draw[color=%s,very thick] " % ("pdg%s" % str(abs(datum["pdg"])))
        out += " -- ".join([f"({t[2]}, {t[1]})" for t in datum["track"]])
        out += ";\n"
   
    legend = set([datum["pdg"] for datum in alldata[en]])
    i = 1
    for pdg in legend:
        if particles[pdg]["name"] != "XXX":
            out +=  "\\draw[color=%s,very thick] (0.25,%d) -- (1.25,%d) node [right,black] {%s};\n" % ("pdg%s" % str(abs(pdg)), 14 - 1.1*i, 14 - 1.1*i, particles[pdg]["name"])
            i += 1

    out += """\\end{tikzpicture}
    \\end{document}
    """

    with open(f"paper/tracks/tracks{en}.tex", "w") as f:
        f.write(out)

    import subprocess

    subprocess.call(
        ["lualatex", f"tracks{en}.tex"], cwd="paper/tracks", stdout=subprocess.DEVNULL
    )
    subprocess.call(
        ["pdfcrop", f"tracks{en}.pdf",  f"tracks{en}.pdf"], cwd="paper/tracks", stdout=subprocess.DEVNULL
    )
    os.remove(f"paper/tracks/tracks{en}.aux")
    os.remove(f"paper/tracks/tracks{en}.log")